In [84]:
import pandas as pd
import pandas as pd
import geopandas as gpd  # Import geopandas
import requests

df = pd.read_csv("education_indicators.csv")

In [85]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 16 columns):
 #   Column                                                                                                               Non-Null Count  Dtype 
---  ------                                                                                                               --------------  ----- 
 0   Time                                                                                                                 1696 non-null   object
 1   Time Code                                                                                                            1694 non-null   object
 2   Country Name                                                                                                         1694 non-null   object
 3   Country Code                                                                                                         1694 non-null   object
 4   Total net enrolmen

In [86]:
# Example: Fill missing numerical values with median
df.fillna(df.median(numeric_only=True), inplace=True)

In [87]:
df.duplicated().sum()
# Remove duplicates
df = df.drop_duplicates()

In [88]:
# Standardize country names
df["country"] = df["Country Name"].str.strip().str.title()

In [89]:
# Load GeoJSON
url = "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json"
geo_data = gpd.read_file(url)

In [90]:
# List of African countries (simplified, adjust as needed)
african_countries = [
    "Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde",
    "Cameroon", "Central African Republic", "Chad", "Comoros", "Congo", "Democratic Republic of the Congo",
    "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia",
    "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", "Libya",
    "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia",
    "Niger", "Nigeria", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone",
    "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda",
    "Zambia", "Zimbabwe"
]

# Filter for African countries
geo_data = geo_data[geo_data["name"].isin(african_countries)]

In [91]:
# Merge datasets
merged_data = geo_data.merge(df, left_on="name", right_on="country", how="left")

In [92]:
# Save cleaned dataset
merged_data.to_csv("clean_education_indicators.csv", index=False)
print("Cleaned dataset saved as 'clean_education_indicators.csv'")

Cleaned dataset saved as 'clean_education_indicators.csv'


In [93]:
# List column names
print("Original column names:", merged_data.columns.tolist())

Original column names: ['id', 'name', 'geometry', 'Time', 'Time Code', 'Country Name', 'Country Code', 'Total net enrolment rate, primary, male (%) [UIS.NERT.1.M]', 'Total net enrolment rate, primary, gender parity index (GPI) [UIS.NERT.1.GPI]', 'Total net enrolment rate, primary, female (%) [UIS.NERT.1.F]', 'Total net enrolment rate, primary, both sexes (%) [UIS.NERT.1]', 'Total net enrolment rate, lower secondary, male (%) [UIS.NERT.2.M]', 'Total net enrolment rate, lower secondary, gender parity index (GPI) [UIS.NERT.2.GPI]', 'Total net enrolment rate, lower secondary, female (%) [UIS.NERT.2.F]', 'Total net enrolment rate, lower secondary, both sexes (%) [UIS.NERT.2]', 'Government expenditure on primary education as % of GDP (%) [UIS.XGDP.1.FSGOV]', 'Government expenditure on secondary education as % of GDP (%) [UIS.XGDP.23.FSGOV]', 'Initial government funding per lower secondary student as a percentage of GDP per capita [UIS.XUNIT.GDPCAP.2.FSGOV]', 'GDP per capita, PPP (current int

In [94]:
rename_dict = {
    'Time': 'Year', 
    'Country Name': 'country',
    'Total net enrolment rate, primary, male (%) [UIS.NERT.1.M]' : 'Primary Male enrollment %', 
    'Total net enrolment rate, primary, gender parity index (GPI) [UIS.NERT.1.GPI]': 'Primary Gender Parity index',
    'Total net enrolment rate, primary, female (%) [UIS.NERT.1.F]': 'Primary female enrollment %',
    'Total net enrolment rate, primary, both sexes (%) [UIS.NERT.1]':'Total Primary enrollment %',
    'Total net enrolment rate, lower secondary, male (%) [UIS.NERT.2.M]': 'Total Secondary male enrollment %',
    'Total net enrolment rate, lower secondary, gender parity index (GPI) [UIS.NERT.2.GPI]': 'Lower secondary Gender Parity index',
    'Total net enrolment rate, lower secondary, female (%) [UIS.NERT.2.F]': 'Total secondary female enrollment %',
    'Total net enrolment rate, lower secondary, both sexes (%) [UIS.NERT.2]':'Total Secondary enrollment %', 
    'Government expenditure on primary education as % of GDP (%) [UIS.XGDP.1.FSGOV]': 'Government primary education expenditure %ge of GDP', 
    'Government expenditure on secondary education as % of GDP (%) [UIS.XGDP.23.FSGOV]': 'Government Secondary education expenditure %ge of GDP',
    'GDP per capita, PPP (current international $) [NY.GDP.PCAP.PP.CD]': 'GDP per capita'
}
merged_data = merged_data.rename(columns=rename_dict)
print("Renamed column names:", merged_data.columns.tolist())

Renamed column names: ['id', 'name', 'geometry', 'Year', 'Time Code', 'country', 'Country Code', 'Primary Male enrollment %', 'Primary Gender Parity index', 'Primary female enrollment %', 'Total Primary enrollment %', 'Total Secondary male enrollment %', 'Lower secondary Gender Parity index', 'Total secondary female enrollment %', 'Total Secondary enrollment %', 'Government primary education expenditure %ge of GDP', 'Government Secondary education expenditure %ge of GDP', 'Initial government funding per lower secondary student as a percentage of GDP per capita [UIS.XUNIT.GDPCAP.2.FSGOV]', 'GDP per capita', 'country']


In [95]:
# Drop one 'country' column, keeping the last occurrence
merged_data = merged_data.loc[:, ~merged_data.columns.duplicated(keep='last')]

# Verify merged_data columns after removing duplicate
print("Merged data columns after removing duplicate 'country':", merged_data.columns.tolist())

Merged data columns after removing duplicate 'country': ['id', 'name', 'geometry', 'Year', 'Time Code', 'Country Code', 'Primary Male enrollment %', 'Primary Gender Parity index', 'Primary female enrollment %', 'Total Primary enrollment %', 'Total Secondary male enrollment %', 'Lower secondary Gender Parity index', 'Total secondary female enrollment %', 'Total Secondary enrollment %', 'Government primary education expenditure %ge of GDP', 'Government Secondary education expenditure %ge of GDP', 'Initial government funding per lower secondary student as a percentage of GDP per capita [UIS.XUNIT.GDPCAP.2.FSGOV]', 'GDP per capita', 'country']


In [96]:
import plotly.express as px
import numpy as np

key_vars = [
    'Primary Male enrollment %', 'Primary female enrollment %',
    'Total Secondary male enrollment %', 'Total secondary female enrollment %',
    'Government primary education expenditure %ge of GDP',
    'Government Secondary education expenditure %ge of GDP', 'GDP per capita'
]
merged_data[key_vars] = merged_data[key_vars].apply(pd.to_numeric, errors='coerce')
merged_data[key_vars] = merged_data[key_vars].replace('..', np.nan)                                                  
print(merged_data[key_vars].describe())



for var in key_vars[:3]:  # Limit to 3 for visualization brevity
    fig = px.histogram(merged_data, x=var, title=f"Distribution of {var}")
    fig.show()

       Primary Male enrollment %  Primary female enrollment %  \
count                 158.000000                   158.000000   
mean                   82.654523                    79.562530   
std                    15.416161                    17.026608   
min                    42.428310                    32.739500   
25%                    71.045990                    63.895715   
50%                    88.704960                    85.100010   
75%                    95.671858                    94.234620   
max                   100.000000                   100.000000   

       Total Secondary male enrollment %  Total secondary female enrollment %  \
count                          76.000000                            76.000000   
mean                           66.057297                            59.360439   
std                            16.261996                            19.782224   
min                            24.002810                            17.542500   
25%      